### Import the library 
Note: An error might popup after this command has exectuted. If it does read it careful before ignoring. 

In [2]:
import psycopg2

### Create a connection to the database

In [3]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

### Next use that connect to get a cursor that we will use to execute queries.

In [4]:
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get cursor to the Database")
    print(e)

#### For this project we will use automactic commit so that each action is commited without having to call conn.commit() after each command. The ability to rollback and commit transactions is a feature of Relational Databases. 

In [5]:
conn.set_session(autocommit=True)

# DROP TABLES

In [7]:
try: 
    cur.execute("DROP TABLE IF EXISTS songplays;")
    cur.execute("DROP TABLE IF EXISTS users;")
    cur.execute("DROP TABLE IF EXISTS songs;")
    cur.execute("DROP TABLE IF EXISTS artists;")
    cur.execute("DROP TABLE IF EXISTS time;")

except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

## Schema for Song Play Analysis

Using the song and log datasets, I'll create a star schema optimized for queries
on song play analysis. This includes the following tables.

<a id="fact"></a>

### Fact Table

1. **songplays** - records in log data associated with song plays, i.e., records with
  page `NextSong`
    - *songplay_id, start_time, user_id, level, song_id, artist_id, session_id,
      location, user_agent*

<a id="dim"></a>

### Dimension Tables

2. **users** - Following information about users:
    - *user_id, first_name, last_name, gender, level*

3. **songs** - Following info about songs:
    - *song_id, title, artist_id, year, duration*

4. **artists** - Artists information:
    - *artist_id, name, location, latitude, longitude*

5. **time** - timestamps of records in **songplays** broken down into specific units
    - *start_time, hour, day, week, month, year, weekday*
    
<img src="images/data-modeling-dbdiagram.png" width="500" height="500">

In order to create these tables, all we need to do is perform some transformation in the data which are already in song_data and log_data directory.

<a id='p_song'></a>

### Create Fact Table songplays

In [8]:
try: 
    cur.execute("""
    
    CREATE TABLE IF NOT EXISTS songplays (
    songplay_id SERIAL PRIMARY KEY,
    start_time bigint NOT NULL,
    user_id int NOT NULL,
    level varchar,
    song_id varchar,
    artist_id varchar,
    session_id int,
    location varchar,
    user_agent varchar
    );
    """)
    
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

### Create Dim Table users

In [9]:
try: 
    cur.execute("""
    
    CREATE TABLE IF NOT EXISTS users (
    user_id int PRIMARY KEY,
    first_name varchar,
    last_name varchar,
    gender varchar,
    level varchar
);
""")
    
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

### Create Dim Table songs

In [10]:
try: 
    cur.execute("""
    
    CREATE TABLE IF NOT EXISTS songs (
    song_id varchar PRIMARY KEY,
    title varchar,
    artist_id varchar,
    year int,
    duration numeric
);
""")
    
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

### Create Dim Table artists

In [11]:
try: 
    cur.execute("""
    
    CREATE TABLE IF NOT EXISTS artists (
    artist_id varchar PRIMARY KEY,
    name varchar,
    location varchar,
    latitude float(7),
    longitude float(7)
);
""")
    
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

### Create Dim Table time

In [12]:
try: 
    cur.execute("""
    
    CREATE TABLE IF NOT EXISTS time (
    start_time bigint PRIMARY KEY,
    hour int,
    day int,
    week int,
    month int,
    year int,
    weekday int
);
""")
    
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

# INSERT RECORDS

## Process Song Data

Perform ETL on the files in *song_data* directory to create two dimensional tables: `songs` table and `artists` table.

This is what a songs file looks like:

<pre>
{"num_songs": 1, "artist_id": "ARD7TVE1187B99BFB1", "artist_latitude": null, "artist_longitude": null, "artist_location": "California - LA", "artist_name": "Casual", "song_id": "SOMZWCG12A8C13C480", "title": "I Didn't Mean To", "duration": 218.93179, "year": 0}
</pre>

For songs table, we'll extract data for songs table by using only the columns corresponding to the songs table suggested in the star schema above. Similarly, we'll select the appropriate columns for artists table.

```python
song_data = df[["song_id", "title", "artist_id", "year", "duration"]].values[0]
song_data
# Looks like this
# array(['SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C', 1982, 233.40363], dtype=object)

artist_data = df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values[0]
artist_data
# Looks like this
# array(['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan], dtype=object)
```